# SARS strains and proteins co-occurence

## Protocol

Goal of this notebook is analysis co-occurence of different types, stains of SARS and SARS-related proteins and genes

We are using fantastic labeled dataset by SiBiteLabs:
https://github.com/SciBiteLabs/CORD19

### To do:
1. Buld dataframe from json-files. Output: data frame with columns ['paper_id','block_id','entity_type_1','..','entity_type_n']. Each row matchs a block. Block is a part of paper's abstract or full_text
2. Group SARS strains. Covid-19 is a main group. Output: lists of strains fo each grooup
3. Filter papers related to any of SARS group and build dataframe. Output: dataframe with columns ['paper_id','title','authors','year','SARS strain','CVPROT']. Each row matchs paper.
4. Plot piechart for strains grous: percent of papers (not blocks) mentioned each strains group. Histogram for strains: top 5 most mentioned strains across the groups.
5. Calculate and plot co-ocurence matrix for strains group and proteins (CVPROT). Calcultae andand plot co-ocurence matrix for top 5 most mentioned strains across and proteins (CVPROT). Use data from the block level(not paper)
6. Plot histogram: number of CVPROT mentioned at block level for SARS in general

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

import re

from tqdm import tqdm

import os
import json
import numpy as np
import collections

## Helper functions

In [5]:
def load_files(dirname):
    filenames = os.listdir(dirname)
    raw_files = []

    for filename in tqdm(filenames):
        filename = dirname + filename
        file = json.load(open(filename, 'rb'))
        raw_files.append(file)
    
    return raw_files


def get_all_files(dirname):
    all_files = []
    
    filenames = os.listdir(dirname)

    for filename in tqdm(filenames):
        filename = dirname + filename
        file = json.load(open(filename, 'rb'))
        all_files.append(file)
    
    return all_files

def get_cat_vocab(cat):
    df_cat = df[cat]
    items = df_cat.dropna().tolist()

    vocab_list = []

    for element in items:
        item = element.split(",")
        for e in item:
            vocab_list.append(e)
    
    c = collections.Counter()

    for word in vocab_list:
        c[word] += 1
        
    result_dic = dict(c)
    
    return result_dic

## 1. Buld dataframe from json-files

In [6]:
#load all files
dirs = [
    'data/scibitelabs/biorxiv_medrxiv/biorxiv_medrxiv/',
    'data/scibitelabs/comm_use_subset/comm_use_subset/',
    'data/scibitelabs/custom_license/custom_license/',
    'data/scibitelabs/noncomm_use_subset/noncomm_use_subset/'    
]

files_stack = []
for dir_ in dirs:
    files = get_all_files(dir_)
    files_stack.append(files)

100%|██████████| 2358/2358 [05:57<00:00,  6.60it/s] 


In [8]:
#build list of entities types
c = collections.Counter()

cat_vocab = []

for files in tqdm(files_stack):
    for file in files:
        for block in file['body_text']:
            dict_file = block['termite_hits'].keys()
            for key in dict_file:
                cat_vocab.append(key)

for word in cat_vocab:
    c[word] += 1
   
vocab_list = (set(list(c.elements())))

100%|██████████| 4/4 [00:49<00:00,  9.42s/it]


In [9]:
vocab_list = sorted(list(vocab_list))
print(vocab_list)

['COUNTRY', 'CVPROT', 'DRUG', 'GENE', 'GOONTOL', 'HPO', 'INDICATION', 'SARSCOV', 'SPECIES']


In [27]:
#build dataframe: entity mentions by blocks ignoring hint count
features = []
for files in tqdm(files_stack):
    for file in files:
        paper_id = file['paper_id']
        
        i = 0
        sections = ['abstract', 'body_text']
        for section in sections:
            for block in file[section]:

                block_id = section + '_' + str(i)
                
                block_features = []
                block_features.append(paper_id)
                block_features.append(block_id)
                
                termite_hits = block['termite_hits']
                
                block_categories = termite_hits.keys()
                block_categories = list(block_categories)
                for cat in vocab_list:
        
                    if cat in block_categories:
                        cat_entities = []
                        for hit in termite_hits[cat]:
                            entity = hit.get('name')
                            if entity not in cat_entities:
                                cat_entities.append(entity)
                                
                        cat_entities = ",".join(cat_entities)

                    else:
                        cat_entities = None

                    block_features.append(cat_entities)

                features.append(block_features)
                i += 1


col_names = ['paper_id', 'block_id']
for cat in vocab_list:
    col_names.append(cat)
df = pd.DataFrame(features, columns=col_names)
df.head()

100%|██████████| 4/4 [01:27<00:00, 17.75s/it]


,paper_id,block_id,COUNTRY,CVPROT,DRUG,GENE,GOONTOL,HPO,INDICATION,SARSCOV,SPECIES
0,f905f78b32f63c6d14a79984dfb33f1b358b8ab4,abstract_0,None,None,None,None,,None,"HIV Infections,Acquired Immunodeficiency Syndrome",retroviral integrase activity,"Virion,HIV-1,Viruses"
1,f905f78b32f63c6d14a79984dfb33f1b358b8ab4,body_text_1,None,None,None,None,"virus maturation,viral replication,establishme...",None,"HIV Infections,Acquired Immunodeficiency Syndrome","retroviral 3' processing activity,retroviral i...","HIV-1,Viruses"
2,f905f78b32f63c6d14a79984dfb33f1b358b8ab4,body_text_2,None,None,None,None,"antigen binding,binding",None,None,None,"Virion,Viruses"
3,f905f78b32f63c6d14a79984dfb33f1b358b8ab4,body_text_3,None,None,Kuwanon L,PC4 and SFRS1 interacting protein 1,binding,None,HIV Infections,retroviral 3' processing activity,"Cats,HIV-1,Viruses"
4,f905f78b32f63c6d14a79984dfb33f1b358b8ab4,body_text_4,None,None,None,None,None,None,None,None,None


In [28]:
#save data
df.to_csv('data/data_ner.csv')

## 2. Group SARS strains

In [73]:
def get_cat_vocab(cat):
    df_cat = df[cat]
    items = df_cat.dropna().tolist()

    vocab_list = []

    for element in items:
        item = element.split(",")
        for e in item:
            vocab_list.append(e)
    
    c = collections.Counter()

    for word in vocab_list:
        c[word] += 1
        
    result_dic = dict(c)
    
    return result_dic

In [74]:
vocab_sars = get_cat_vocab('SARSCOV')

In [75]:
vocab_sars

{'retroviral integrase activity': 554,
 "retroviral 3' processing activity": 24,
 'viral life cycle': 20547,
 'virion maturation': 355,
 'transmission of virus': 44020,
 'Severe acute respiratory syndrome coronavirus 2': 9419,
 'incubation period': 18425,
 'virus receptor activity': 2101,
 'viral release from host cell': 3860,
 'SARS coronavirus': 25300,
 'viral genome': 8785,
 'transport of virus': 878,
 'human-to-human viral transmission': 2505,
 'viral nucleocapsid': 4590,
 'virion attachment to host cell': 123,
 'modulation by virus of host morphology or physiology': 302,
 'viral process': 166,
 'viral membrane': 1398,
 'virion membrane': 1398,
 'virus maturation': 339,
 'viral strain': 4857,
 'viral gene expression': 717,
 'virion assembly': 1912,
 'SARS-CoV genome': 608,
 'viral transcription': 524,
 'SARS coronavirus NS-1': 2803,
 'SARS coronavirus Tor2': 254,
 'RNA viral genome': 312,
 'viral translation': 879,
 '': 1313,
 'viral capsid': 1318,
 'viral entry into host cell': 49

In [ ]:
sars = ['SARS coronavirus']
covid_19 = ['Severe acute respiratory syndrome coronavirus 2']

sars_sin_strains = [
    'SARS coronavirus Sin2748',
    'SARS coronavirus Sin2774',
    'SARS coronavirus Sin3725V',
    'SARS coronavirus Sin0409',
    'SARS coronavirus Sin_WNV',
    'SARS coronavirus Sin2500',
    'SARS coronavirus Sin2677',
    'SARS coronavirus Sin2679',
    'SARS coronavirus Sin846',
    'SARS coronavirus Sin847',
    'SARS coronavirus Sin842',
    'SARS coronavirus Sin845',
    'SARS coronavirus Sin852',
    'SARS coronavirus Sin848',
    'SARS coronavirus Sin850',
    'SARS coronavirus Sin849',
    'SARS coronavirus Sin3408',
    'SARS coronavirus SinP2',
    'SARS coronavirus Sin3408L',
    'SARS coronavirus SinP5',
    'SARS coronavirus SinP3',
    'SARS coronavirus SinP4',
]

sars_betacov_strains = [
     'BtRf-BetaCoV/JL2012',
     'BtRf-BetaCoV/SX2013',
     'BtRf-BetaCoV/HeB2013',
]

sars_tw_strains = [
    'SARS coronavirus TW1',
    'SARS coronavirus TW2',
    'SARS coronavirus TW4',
    'SARS coronavirus TW5',
    'SARS coronavirus TW10',
    'SARS coronavirus TWC2',
    'SARS coronavirus TWC3',
    'SARS coronavirus TW9',
    'SARS coronavirus TW8',
    'SARS coronavirus TW7',
    'SARS coronavirus TW6',
    'SARS coronavirus TW3',
    'SARS coronavirus TW4',
]

sars_shanghai_strains = [
    'SARS coronavirus ShanghaiQXC1',
    'SARS coronavirus ShanghaiQXC2',
]

sars_gz_strains = [
    'SARS coronavirus GZ02',
    'SARS coronavirus GZ-C',
    'SARS coronavirus GZ-B',
    'SARS coronavirus GZ50',
    'SARS coronavirus GZ0402',
]

sars_bj_stains = [
    'SARS coronavirus BJ04',
    'SARS coronavirus BJ302',
    'SARS coronavirus BJ01',
    'SARS coronavirus BJ182-12',
    'SARS coronavirus BJ02',
    'SARS coronavirus BJ03',
    'SARS coronavirus BJ202',
]

sars_lc_stains = [
    'SARS coronavirus LC3',
    'SARS coronavirus LC2',
    'SARS coronavirus LC5',
    'SARS coronavirus LC1',
]

sars_other_strains = [
    'SARS coronavirus NS-1',
    'SARS coronavirus Tor2',
    'Bat SARS-like coronavirus',
    'SARS coronavirus Urbani',
    'SARS coronavirus CUHK-W1',
    'SARS coronavirus MA15',
    'SARS coronavirus ZS-C',
    'SARS coronavirus Sino1-11',
    'SARS coronavirus HSR 1',
    'Bat SARS-like coronavirus WIV1',
    'SARS coronavirus ZJ01',
    'SARS coronavirus Frankfurt 1',
    'SARS coronavirus HC/SZ/61/03',
    'SARS coronavirus AS',
    'SARS coronavirus GD03T0013',
    'SARS coronavirus GD01': 71,
    'SARS coronavirus HKU-39849',
    'SARS coronavirus CUHK-AG01',
    'SARS coronavirus HZS2-Fb',
    'SARS coronavirus PUMC01',
    'SARS coronavirus B012',
    'SARS coronavirus ExoN1',
    'SARS coronavirus C025',
    'SARS coronavirus PUMC03',
    'SARS coronavirus wtic-MB',
    'SARS coronavirus HSZ-Cb',
    'SARS coronavirus A022',
    'SARS coronavirus SZ1',
    'SARS coronavirus WH20',
    'SARS coronavirus SoD',
    'SARS coronavirus BJ01': 174,
 'SARS coronavirus LC4': 5,
 'SARS coronavirus JMD': 1,
 'SARS coronavirus ES191': 1,
 'SARS coronavirus PUMC02': 7,
 'SARS coronavirus Taiwan TC2': 1,
 'SARS coronavirus Taiwan TC3': 1,
 'SARS coronavirus Taiwan TC1': 3,
 'SARS coronavirus ZMY 1': 15,
 'SARS coronavirus GZ43': 17,
 'SARS coronavirus SZ13': 7,
 'SARS coronavirus CUHK-L2': 1,
 'SARS coronavirus HSZ-A': 1,
 'SARS coronavirus HKU-65806': 1,
 'SARS coronavirus ZS-B': 4,
 'SARS coronavirus GD69': 6,
 'SARS coronavirus TW11': 5,
 'SARS coronavirus HGZ8L1-A': 4,
 'SARS coronavirus Sino3-11': 4,
 'SARS coronavirus CUHK-AG02': 5,
 'SARS coronavirus CUHK-AG03': 5,
 'SARS coronavirus LLJ-2004': 2,
 'SARS coronavirus GZ60': 11,
 'SARS coronavirus Rs_672/2006': 1,
 'SARS coronavirus GZ0401': 2,
 'SARS coronavirus HZS2-C': 3,
 'SARS coronavirus HZS2-Fc': 3,
 'SARS coronavirus GZ-A': 1,
 'SARS Coronavirus CDC#200301157': 2,
 'SARS coronavirus A030': 2,
 'SARS coronavirus A013': 1,
 'SARS coronavirus B039': 1,
 'SARS coronavirus PC4-227': 3,
 'SARS coronavirus PC4-136': 2,
 'SARS coronavirus civet020': 2,
 'SARS coronavirus civet010': 1,
 'SARS coronavirus PC4-13': 2,
 'SARS coronavirus HSZ-Bc': 3,
 'SARS coronavirus HSZ-Bb': 3,
 'SARS coronavirus HSZ-Cc': 3,
 'SARS coronavirus C028': 2,
 'SARS coronavirus A001': 1,
 'SARS coronavirus B024': 1,
 'SARS coronavirus civet014': 1,
 'SARS coronavirus ZS-A': 4,
 'SARS coronavirus TWC': 1,
 'SARS coronavirus PC4-115': 1,
 'SARS coronavirus HSZ2-A': 2,
 'SARS coronavirus HGZ8L2': 2,
 'SARS coronavirus HZS2-E': 2,
 'SARS coronavirus HZS2-D': 2
]

In [77]:
vocab_cvprot = get_cat_vocab('CVPROT')

In [78]:
vocab_cvprot

{'SPIKE_WCPV': 18610,
 'R1AB_WCPV': 20633,
 'NCAP_WCPV': 11323,
 'R1A_WCPV': 10141,
 'NS8_WCPV': 558,
 'VME1_WCPV': 7109,
 'AP3A_WCPV': 582,
 'VEMP_WCPV': 5832,
 'A0A663DJA2_9BETC': 59,
 'ORF9B_WCPV': 182,
 'Y14_WCPV': 35,
 'NS6_WCPV': 498,
 '': 933,
 'NS7B_WCPV': 132,
 'NS7A_WCPV': 113}

In [80]:
vocab_genes = get_cat_vocab('GENE')

In [82]:
vocab_genes

{'PC4 and SFRS1 interacting protein 1': 24,
 'ATP/GTP binding protein like 1': 90,
 'double homeobox 4': 61,
 'tumor protein p53': 1734,
 'structural maintenance of chromosomes flexible hinge domain containing 1': 4,
 'paired box 7': 15,
 'paired box 3': 11,
 'MDM2 proto-oncogene': 228,
 'tripartite motif containing 24': 21,
 'platelet derived growth factor receptor alpha': 100,
 'integrin subunit alpha 7': 11,
 'matrix metallopeptidase 8': 86,
 'fibroblast growth factor 2': 290,
 'angiotensin I converting enzyme 2': 4253,
 'furin': 1142,
 ' paired basic amino acid cleaving enzyme': 1142,
 'CD4 molecule': 8998,
 'angiotensin I converting enzyme': 820,
 'C-reactive protein': 2924,
 '': 2370,
 'nucleophosmin 1': 178,
 'exportin 1': 254,
 'heterogeneous nuclear ribonucleoprotein A1': 232,
 'nuclear factor kappa B subunit 1': 1763,
 'dipeptidyl peptidase 4': 2134,
 'inhibitor of nuclear factor kappa B kinase subunit beta': 231,
 'glycogen synthase kinase 3 beta': 164,
 'SMAD family member 

In [83]:
vocab_hpo = get_cat_vocab('HPO')

In [84]:
vocab_hpo

{'Pneumonia': 21615,
 'Myopathy': 391,
 '': 31364,
 'Increased reactive oxygen species production': 2214,
 'Alopecia': 915,
 'Palpebral edema': 69,
 'Papilloma': 492,
 'T-cell lymphoma': 280,
 'Recurrent lower respiratory tract infections': 2032,
 'Diarrhea': 21696,
 'Intermittent diarrhea': 79,
 'Respiratory tract infection': 9935,
 'Allergy': 6174,
 'Fever': 20396,
 'Elevated hepatic transaminase': 552,
 'Abnormal liver physiology': 59,
 'Cough': 11009,
 'Breathing dysregulation': 206,
 'Hepatic steatosis': 510,
 'Nonproductive cough': 788,
 'Elevated C-reactive protein level': 193,
 'Myalgia': 2530,
 'Inflammation of the large intestine': 981,
 'Hypertension': 2561,
 "Crohn's disease": 463,
 'Respiratory failure': 2603,
 'Autoimmunity': 6234,
 'Systemic lupus erythematosus': 667,
 'Rheumatoid arthritis': 1121,
 'Breast carcinoma': 1595,
 'Increased inflammatory response': 142,
 'Fatigue': 1896,
 'Dyspnea': 4480,
 'Phenotypic abnormality': 572,
 'Joint swelling': 63,
 'Conjunctival h

## 3. Filter Papers, Create DataFrame

In [ ]:
#https://www.kaggle.com/maksimeren/covid-19-literature-clustering


In [ ]:
data = df.fillna('qqqqq')

In [ ]:
#vocab_cvprot per step 2:

vocab_cvprot = ['SPIKE_WCPV', 'R1AB_WCPV',
 'NCAP_WCPV',
 'R1A_WCPV',
 'NS8_WCPV',
 'VME1_WCPV',
 'AP3A_WCPV',
 'VEMP_WCPV',
 'A0A663DJA2_9BETC',
 'ORF9B_WCPV',
 'Y14_WCPV',
 'NS6_WCPV',
  'NS7B_WCPV',
 'NS7A_WCPV']


In [ ]:
sars_paper_id = {'PAPER_ID': [], 'SARS_COV': [],'SPECIES': [],'CVPROT': []}
for i in range(0, data.shape[0]):
  row = data.iloc[i]
  prot_field = row[3]
  if prot_field not in vocab_cvprot: continue
  paper_id = row[0]
  SARS_cov = row[9]
  species = row[10]
  sars_paper_id['PAPER_ID'].append(paper_id)
  sars_paper_id['SARS_COV'].append(SARS_cov)
  sars_paper_id['SPECIES'].append(species)
  sars_paper_id['CVPROT'].append(prot_field)
  
df_cvprot = pd.DataFrame(sars_paper_id, columns=['PAPER_ID', 'SARS_COV', 'SPECIES', 'CVPROT'])
df_cvprot.head()

## 4. Create Pie Chart, Histogram

In [ ]:
#check data unique relation - paper ID & protein
df_paper_id = df_cvprot["PAPER_ID"]
df_prot = df_cvprot["CVPROT"]
df_check_unq = pd.concat([df_paper_id, df_prot], axis=1)

In [ ]:
df_check_unq.shape
df_check_unq = df_check_unq.drop_duplicates()
df_check_unq.shape

In [ ]:
#Plot piechart for strains groups: percent of papers (not blocks) mentioned each strains group. 
#IMPORTANT: STRAINS or PROTEINS? This is proteins.
#get count of papers that contain at least one mention of the protein
prot_counts = df_check_unq.groupby('CVPROT').CVPROT.value_counts()
prot_counts

In [ ]:
#get total paper count
papers = df_cvprot["PAPER_ID"]
papers.shape[0]
papers = papers.drop_duplicates()
papers.shape[0]
total_papers = papers.shape[0]

In [ ]:
#create %s
A0A663DJA2_9BETC = prot_counts[0]
AP3A_WCPV = prot_counts[1]
NCAP_WCPV = prot_counts[2]
NS6_WCPV = prot_counts[3]
NS7A_WCPV = prot_counts[4]
NS7B_WCPV = prot_counts[5]
ORF9B_WCPV = prot_counts[6]
R1A_WCPV = prot_counts[7]
R1AB_WCPV = prot_counts[8]
SPIKE_WCPV = prot_counts[9]
VEMP_WCPV = prot_counts[10]
VME1_WCPV = prot_counts[11]
Y14_WCPV = prot_counts[12]

In [ ]:
SPIKE_WCPV_pt = round(SPIKE_WCPV/total_papers,2)
R1AB_WCPV_pt = round(R1AB_WCPV/total_papers,2)
NCAP_WCPV_pt = round(NCAP_WCPV/total_papers,2)
R1A_WCPV_pt = round(R1A_WCPV/total_papers,2)
#NS8_WCPV_pt = round(NS8_WCPV/total_papers,2)
VME1_WCPV_pt = round(VME1_WCPV/total_papers,2)
AP3A_WCPV_pt = round(AP3A_WCPV/total_papers,2)
VEMP_WCPV_pt = round(VEMP_WCPV/total_papers,2)
A0A663DJA2_9BETC_pt = round(A0A663DJA2_9BETC/total_papers,2)
ORF9B_WCPV_pt = round(ORF9B_WCPV/total_papers,2)
Y14_WCPV_pt = round(Y14_WCPV/total_papers,2)
NS6_WCPV_pt = round(NS6_WCPV/total_papers,2)
NS7B_WCPV_pt = round(NS7B_WCPV/total_papers,2)
NS7A_WCPV_pt = round(NS7A_WCPV/total_papers,2)

In [ ]:
print(SPIKE_WCPV_pt,R1AB_WCPV_pt,NCAP_WCPV_pt,R1A_WCPV_pt,NS8_WCPV_pt,VME1_WCPV_pt,AP3A_WCPV_pt,VEMP_WCPV_pt,A0A663DJA2_9BETC_pt,ORF9B_WCPV_pt, Y14_WCPV_pt,NS6_WCPV_pt,NS7B_WCPV_pt,NS7A_WCPV_pt)

In [ ]:
#Pie Chart - Adjust labels, sizes & explode after running on full data
#https://matplotlib.org/3.1.1/gallery/pie_and_polar_charts/pie_features.html

In [ ]:
import matplotlib.pyplot as plt

# Pie chart, where the slices will be ordered and plotted counter-clockwise:
labels = 'SPIKE_WCPV_pt','R1A_WCPV_pt','NCAP_WCPV_pt','VME1_WCPV_pt','VEMP_WCPV_pt','Others'
sizes = [0.38,0.32,0.27,0.21,0.16,0.02]
explode = (0.1, 0.0, 0, 0,0,0)  # only "explode" the 2nd slice (i.e. 'Hogs')

fig1, ax1 = plt.subplots()
ax1.pie(sizes, explode=explode, labels=labels, autopct='%1.1f%%',
        shadow=True, startangle=90)
ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
fig = plt.gcf()
fig.set_size_inches(8,8)
plt.show()


In [ ]:
#Do the same for strains?


In [ ]:
#Histogram for strains: top 5 most mentioned strains across the groups (includes multiple counts in one paper)

#IMPORTANT: STRAINS or PROTEINS???
#This is proteins.


In [ ]:
cvprot_counts = df_cvprot['CVPROT'].value_counts()

In [ ]:
cvprot_counts
top_5 = cvprot_counts[0:5]
top_5
top_5.plot(kind='bar')
plt.title('Top 5 Protein Mentions')
plt.xlabel('value')
plt.ylabel('protein')


In [ ]:
#Do the same for strains?


## 4. Get Strain Count Data, Create HeatMaps of Strains

In [ ]:
#https://www.kaggle.com/rtatman/co-occurrence-matrix-plot-in-python

In [ ]:
import seaborn as sns

In [ ]:
df_cvprot.shape[0], df_cvprot.PAPER_ID.nunique()  # <check the input data is block level. 


In [ ]:
#function to create counts dataframe
def df_co_occurrance(df, strain_group):
  strains_df = df.copy()  
  for i in strain_group:
        eval_match = df.SARS_COV.str.contains(i)
        strains_df[i] = eval_match
  return strains_df

In [ ]:
# repeat for all strain groups
strains_df = df_co_occurrance(df_cvprot,sars_sin_strains)

In [ ]:
#transform data and create heatmap (repeat for all strain groups)
strains_df.iloc[:,4:] = strains_df.iloc[:,4:].astype(int) 
strains_df = strains_df.iloc[:,3:]
prot_strains_grpd = strains_df.groupby('CVPROT').sum()
sns.heatmap(prot_strains_grpd)